# Conversione dei dati di PA Digitale in LD

In [1]:
import pandas as pd
import rdflib
import json
from IPython.display import Markdown as md
from IPython.display import JSON

In [22]:
from pybars import Compiler
import urllib.parse

# Funzione helper per evitare di ottenere IRI non valide
def encodeURIComponent(this,value):
    return urllib.parse.quote(value)

def extractNumeroMisura(this, value):
    ret=None
    match = re.match(r"(\d+\.\d+\.\d+)", value)

    if match:
        ret = match.group(1)

    return ret

def apply_handlebars_template(template, data):
    compiler = Compiler()
    context = {
        'encodeURIComponent': encodeURIComponent,
        'extractNumeroMisura': extractNumeroMisura
    }
    return compiler.compile(template)(data, helpers=context)


In [3]:
with open("srcdata/data/candidature_comuni_finanziate.json", 'r') as f:
    PADigitale_json=json.load(f)

Per i dati di PADigitale le ontologie di riferimento sono:

* L'ontologia delle organizzazioni https://w3id.org/italia/onto/COV/
* L'ontologia dei luoghi https://w3id.org/italia/onto/Project/
    

In [5]:
md("All'interno del file ci sono {} candidature finanziate".format(len(PADigitale_json["candidature"])))

All'interno del file ci sono 568 candidature finanziate

Il formato di partenza dei dati di PADigitale è:

In [7]:
JSON(PADigitale_json["candidature"][0])

<IPython.core.display.JSON object>

Definiamo quindi un template per PADigitale

Notare che abbiamo usato l'iterazione all'interno del template e il costrutto `#unless` che permette di generare un JSON-LD valido.

In [37]:
PADigitale_template="""
{
    "@context": {
    "l0": "https://w3id.org/italia/onto/l0/",
    "CLV": "https://w3id.org/italia/onto/CLV/",
    "COV": "https://w3id.org/italia/onto/COV/",
    "PRJ": "https://w3id.org/italia/onto/Project/"
    },
    "@graph": [
        {{#each .candidature}}
        {
        "@type": "PRJ:PublicInvestmentProject",
        "@id": "https://w3id.org/italia/data/PublicInvestmentProject/{{encodeURIComponent codice_cup}}",

        "PRJ:uniqueProjectCodeValue": "{{codice_cup}}",
        "PRJ:projectTitle": "Progetto {{codice_cup}} per il comune {{codice_ipa}} relativo all'avviso {{avviso}}",
        "PRJ:hasCall": {
            "@id":"https://w3id.org/italia/data/Call/{{encodeURIComponent avviso}}",
            "l0:name": "{{avviso}}",
            "@type": "PRJ:Call"
            },
        "PRJ:hasProgramme": {
            "@type": "PRJ:Programme",
            "l0:name": "PNRR"
            }
        }
        {{#unless @last}},{{/unless}}
        {{/each}}
    ]
}
"""

In [38]:
# Trasformazione di JSON in JSON-LD

PADigitale_json_ld = apply_handlebars_template(PADigitale_template,PADigitale_json)

In [39]:
with open ("LD/json-ld/candidature_comuni_finanziate-ld.json", "w+") as f:
    f.write(PADigitale_json_ld)

In [40]:
print(PADigitale_json_ld)


{
    "@context": {
    "l0": "https://w3id.org/italia/onto/l0/",
    "CLV": "https://w3id.org/italia/onto/CLV/",
    "COV": "https://w3id.org/italia/onto/COV/",
    "PRJ": "https://w3id.org/italia/onto/Project/"
    },
    "@graph": [
        {
        "@type": "PRJ:PublicInvestmentProject",
        "@id": "https://w3id.org/italia/data/PublicInvestmentProject/D81F22000260006",

        "PRJ:uniqueProjectCodeValue": "D81F22000260006",
        "PRJ:projectTitle": "Progetto D81F22000260006 per il comune c_g039 relativo all'avviso 1.4.4 Adozione SPID CIE  - Comuni - Aprile 2022",
        "PRJ:hasCall": {
            "@id":"https://w3id.org/italia/data/Call/1.4.4%20Adozione%20SPID%20CIE%20%20-%20Comuni%20-%20Aprile%202022",
            "l0:name": "1.4.4 Adozione SPID CIE  - Comuni - Aprile 2022",
            "@type": "PRJ:Call"
            },
        "PRJ:hasProgramme": {
            "@type": "PRJ:Programme",
            "l0:name": "PNRR"
            }
        }
        ,
        {
      

In [41]:
PADigitale_g = rdflib.Graph()
PADigitale_g.parse(data=PADigitale_json_ld,format="json-ld");

In [17]:
query="""
SELECT (COUNT(DISTINCT ?CUP) as ?num_CUP) 
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:uniqueProjectCodeValue ?CUP
}
"""

res=opencup_g.query(query)

for row in res:
    print(row)

(rdflib.term.Literal('538', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


Come si vede, i CUP sono in numero inferiore, in quanto ve ne sono di duplicati all'interno del file.

Esempio di query: qual è il totale di costo dei progetti per ciascun ente?

In [24]:
query="""
SELECT ?funder (SUM(?totalCost) AS ?funderTotalCost)
WHERE {
    ?prj a PRJ:PublicInvestmentProject;
    PRJ:projectFunder ?funder;
    PRJ:projectTotalCost ?totalCost.
}
GROUP BY ?funder
ORDER BY DESC(?funderTotalCost)
"""

res=opencup_g.query(query)

for row in res:
    print(row)

(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00339370272'), rdflib.term.Literal('7127677', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/80015010723'), rdflib.term.Literal('4858680', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00297110389'), rdflib.term.Literal('1692369', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00080270838'), rdflib.term.Literal('1547897', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/PublicOrganization/00229080338'), rdflib.term.Literal('1479027', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://w3id.org/italia/data/Publi

## Dati di IndicePA

I dati di IndicePA ci occorrono, ad esempio, per collegare le partite IVA/Codici Fiscali agli enti